In [ ]:
import pandas as pd
import chromadb
import numpy as np
import requests
from langchain.agents import Tool, initialize_agent
from langchain_openai import OpenAI
import openai
from langchain.prompts import PromptTemplate
from langchain.agents import AgentType
import os

# Accede a la clave API desde una variable de entorno
openai_api_key = os.getenv("OPENAI_API_KEY")

# Configura tu cliente con esta clave
openai.api_key = openai_api_key


# Carga el archivo CSV
noticias_df = pd.read_csv("./hidrogeno_verde.csv")

# Fecha en formato datetime
noticias_df['date'] = pd.to_datetime(noticias_df['date'], format='%b %d, %Y @ %H:%M:%S.%f')
noticias_df['date'] = noticias_df['date'].dt.strftime('%Y-%m-%d')

In [2]:
display(noticias_df.head())

,date,id_news,text,title,url,polarity_score,polarity_label,media_outlet,country
0,2024-10-19,54.272.642,"Santiago, octubre de 2024 – Como parte de la c...",Ausenco impulsa la transición energética a tra...,https://tierramarillano.cl/2024/10/20/ausenco-...,-,-,tierraamarillano,chile
1,2024-10-17,54.270.980,"Claudio Seebach, una voz respetada en el sect...","Claudio Seebach aterriza la ""realidad"" del hid...",https://www.emol.com/noticias/Economia/2024/10...,-,-,emol,chile
2,2024-10-16,54.274.158,"En septiembre, el Presidente Gabriel Boric i...",Hidrógeno verde: La industria estratégica para...,https://www.emol.com/noticias/Economia/2024/10...,-,-,emol,chile
3,2024-10-14,54.293.366,– Eliana Ximena Oyarzún Nancuante La División ...,Subpesca solicita a la industria del hidrógeno...,https://laprensaaustral.cl/2024/10/15/subpesca...,-,-,laprensaaustral,chile
4,2024-10-14,54.293.490,– Eliana Ximena Oyarzún Nancuante Su profunda ...,Sindicato de Trabajadores Enap llama a “aterri...,https://laprensaaustral.cl/2024/10/15/sindicat...,-,-,laprensaaustral,chile


In [ ]:
import chromadb
from langchain.embeddings import OpenAIEmbeddings


embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Configurar ChromaDB
client = chromadb.PersistentClient(path=".")
collection = client.get_or_create_collection(name="hidrogeno_verde")

for _, row in noticias_df.iterrows():
    contenido = row["title"] + row["text"] 
    metadata = {'media_outlet': row['media_outlet'],"title": row["title"], "date": str(row["date"]), 'url': row['url']}
    embedding = embeddings.embed_query(contenido)

    collection.add(
        documents=[contenido],
        embeddings=[embedding],
        metadatas=[metadata],
        ids=[str(row.name)])



In [ ]:
# Función para realizar la búsqueda en ChromaDB
def search_in_chromadb(query):
    # Realiza la búsqueda en ChromaDB utilizando la consulta
    results = doc_collection.query(
        query_embeddings=embeddings.embed_query(query),
        n_results=3  # Número de resultados a obtener
    )
    return results

In [ ]:
# Ingresar consulta
query = "poner el texto aqui"
print(search_in_chromadb(query))


In [ ]:
# Función para búsqueda en ChromaDB (o cualquier otra base de datos)
def search_tool(query):
    search_results = search_in_chromadb(query)
    return "\n".join([str(doc) for doc in search_results['documents']])

# Función para búsqueda en Wikipedia
def wikipedia_search(query):
    response = requests.get(f"https://es.wikipedia.org/w/api.php", params={
        "action": "query",
        "format": "json",
        "titles": query,
        "prop": "extracts",
        "exintro": True
    })
    pages = response.json()['query']['pages']
    page = next(iter(pages.values()))
    return page.get('extract', "No se encontró información en Wikipedia.")

# Inicializar el LLM
llm = OpenAI(temperature=0.7, openai_api_key=openai_api_key)

# Crear herramientas
tools = [
    Tool(
        name="ChromaDB Search",
        func=search_tool,
        description="Busca en la base de datos de ChromaDB los documentos relevantes."
    ),
    Tool(
        name="Wikipedia Search",
        func=wikipedia_search,
        description="Consulta Wikipedia para obtener más información sobre el tema."
    )
]

# Definir el prompt usando PromptTemplate
prompt_template = """
Eres un agente que puede interactuar con herramientas para obtener información relevante para responder preguntas. 
Tienes acceso a dos herramientas:

1. **ChromaDB Search**: Busca en una base de datos de documentos para obtener información relevante.
2. **Wikipedia Search**: Realiza una consulta a Wikipedia para obtener un resumen de un tema.

Cuando recibas una pregunta, debes decidir qué herramienta usar y cómo responder de manera concisa y precisa. 
Si es posible, utiliza primero la búsqueda en ChromaDB, si no encuentras suficiente información, consulta Wikipedia.
"""

# Crear el PromptTemplate
prompt = PromptTemplate(input_variables=["tools", "query", "agent_scratchpad", "tool_names"], template=prompt_template)

# Crear el agente React usando `initialize_agent` (asegurando que pasemos los argumentos correctamente)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Ejecutar el agente con una consulta usando `invoke`
query = ""
response = agent.invoke(query)

# Mostrar la respuesta
print(response)

